In [1]:
import habitat
from typing import Any
from gym import spaces
import habitat_sim
import math

import numpy as np
import random
import quaternion

from PIL import Image
from habitat_sim.utils.common import d3_40_colors_rgb
import numpy as np
import os

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
class AgentPosition:
    angle_weight = 12
    coordinate_weight = 4
    min_diff = 2
    
    def __init__(self, position, rotation):
        self.position = position
        self.rotation = rotation
        
    def get_position(self):
        return self.position
    
    def get_rotation(self):
        return self.rotation
        
    def radian_to_grade(self, angle):
        return angle / 2 / math.pi * 360 + 180
        
    def print(self):
        print('Position= ', self.position, sep='')
        print('Rotation=', self.rotation, sep='')
        
    
    @staticmethod
    def get_difference_position(p1, p2):
        return math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2 + (p1[2] - p2[2])**2)
    
    @staticmethod
    def get_difference_rotation(r1, r2):
        f1 = quaternion.as_float_array(r1)[0]
        f2 = quaternion.as_float_array(r1)[2]
        
        s1 = quaternion.as_float_array(r2)[0]
        s2 = quaternion.as_float_array(r2)[2]
        return math.sqrt((f1 - s1) ** 2 + (f2 - s2) ** 2)
    
    @staticmethod
    def get_diff(p1, p2):
        position1 = p1.get_position()
        position2 = p2.get_position()
        position_diff = AgentPosition.get_difference_position(position1, position2)
        
        rotation1 = p1.get_rotation()
        rotation2 = p2.get_rotation()
        
        rotation_diff = AgentPosition.get_difference_rotation(rotation1, rotation2)
        
        return position_diff * AgentPosition.coordinate_weight + rotation_diff * AgentPosition.angle_weight
        
    @staticmethod
    def is_close(p1, p2):
        
        diff = AgentPosition.get_diff(p1, p2)
        
#         print(diff)
        
        return AgentPosition.min_diff > diff
    
    def create_np_array(self):
        return np.array([
            self.position[0] * AgentPosition.coordinate_weight,
            self.position[1] * AgentPosition.coordinate_weight,
            self.position[2] * AgentPosition.coordinate_weight,
            quaternion.as_float_array(self.rotation)[0] * AgentPosition.angle_weight,
            quaternion.as_float_array(self.rotation)[2] * AgentPosition.angle_weight
        ])
        
        


In [3]:

# Define the sensor and register it with habitat
# For the sensor, we will register it with a custom name
@habitat.registry.register_sensor(name="my_supercool_sensor")
class AgentPositionSensor(habitat.Sensor):
    def __init__(self, sim, config, **kwargs: Any):
        super().__init__(config=config)

        self._sim = sim
        # Prints out the answer to life on init
        print("The answer to life is", self.config.ANSWER_TO_LIFE)

    # Defines the name of the sensor in the sensor suite dictionary
    def _get_uuid(self, *args: Any, **kwargs: Any):
        return "agent_position"

    # Defines the type of the sensor
    def _get_sensor_type(self, *args: Any, **kwargs: Any):
        return habitat.SensorTypes.POSITION

    # Defines the size and range of the observations of the sensor
    def _get_observation_space(self, *args: Any, **kwargs: Any):
        return spaces.Box(
            low=np.finfo(np.float32).min,
            high=np.finfo(np.float32).max,
            shape=(3,),
            dtype=np.float32,
        )

    # This is called whenver reset is called or an action is taken
    def get_observation(
        self, observations, *args: Any, episode, **kwargs: Any
    ):
        return AgentPosition(self._sim.get_agent_state().position, self._sim.get_agent_state().rotation).create_np_array()

In [4]:
config = habitat.get_config()
config.defrost()
config.DATASET.DATA_PATH = "/home/green-tea/all_projects/coursework_3/data/datasets/pointnav/habitat-test-scenes/v1/{split}/{split}.json.gz"
config.SIMULATOR.AGENT_0.SENSORS = ['RGB_SENSOR', 'DEPTH_SENSOR']

config.SIMULATOR.RGB_SENSOR.HEIGHT = 256
config.SIMULATOR.RGB_SENSOR.WIDTH = 256

config.SIMULATOR.DEPTH_SENSOR.HEIGHT = 256
config.SIMULATOR.DEPTH_SENSOR.WIDTH = 256

del config.SIMULATOR['SEMANTIC_SENSOR']

config.TASK.AGENT_POSITION_SENSOR = habitat.Config()
config.TASK.AGENT_POSITION_SENSOR.TYPE = "my_supercool_sensor"
config.TASK.AGENT_POSITION_SENSOR.ANSWER_TO_LIFE = 42
config.TASK.SENSORS.append("AGENT_POSITION_SENSOR")

config.freeze()

In [5]:
env = habitat.Env(config=config)

2020-01-30 16:38:30,471 Initializing dataset PointNav-v1
2020-01-30 16:38:32,189 initializing sim Sim-v0
I0130 16:38:36.150939 5600 simulator.py:127] Loaded navmesh data/scene_datasets/gibson/Kerrtown.navmesh
2020-01-30 16:38:36,157 Initializing task Nav-v0


The answer to life is 42


In [6]:
action_mapping = {
    1: 'MOVE_FORWARD',
    2: 'TURN_LEFT',
    3: 'TURN_RIGHT'
}

In [7]:
def check_is_good(image, env):
    
#     value = env._sim.distance_to_closest_obstacle(env._sim.get_agent_state().position, np.inf)
#     print(value, flush=True)
    
#     return value > 0.1 and value < np.inf 

    ans = 0
    count = 0
    for el in image:
        for sub_el in el:
            count += 1
            if np.all(sub_el == 0):
                ans += 1
                
    q = (ans / count) < 0.3
    
    if q:
        print('GOOD!', flush=True)
    else:
        print('Terrible', flush=True)
        print(env.current_episode, flush=True)
        
    return q
    

In [8]:
def check_action(cur_action, last_action, is_bump):
    if is_bump and cur_action == 'MOVE_FORWARD':
        return False
    
    if cur_action == 'TURN_LEFT' and last_action == 'TURN_RIGHT':
        return False
    if cur_action == 'TURN_RIGHT' and last_action == 'TURN_LEFT':
        return False

    return True

In [9]:
def get_random_action(last, is_bump):
    action = random.choice(list(action_mapping))

    while not(check_action(action, last, is_bump)):
        action = random.choice(list(action_mapping))

    return action

In [10]:
class Saver:
    def __init__(self):
        self.count = 0
        self.path = 'images'
        self.all_positions = {}
        
    def save_image(self, observations, env):
        cur_scene = env.current_episode.scene_id
        
        if self.check_exist_image(observations, cur_scene):
            return;
        
        rgb_image = Image.fromarray(observations['rgb'], mode="RGB")

        depth_image = Image.fromarray((np.squeeze(observations['depth']) * 255).astype(np.uint8), mode="L")

        rgb_image.save(self.path + "/rgb/rgb_image_" + str(self.count) + ".png")
        depth_image.save(self.path + "/depth/depth_image_" + str(self.count) +".png")

        self.count += 1
        
        if cur_scene in self.all_positions:
            self.all_positions[cur_scene] = np.append(self.all_positions[cur_scene], [observations['agent_position']], axis=0)
        else:
            self.all_positions[cur_scene] = np.array([observations['agent_position']])
        
    def check_exist_image(self, observations, cur_scene):
        cur_position = observations['agent_position']
        
        if not(cur_scene in self.all_positions):
            return False
        
        min_diff = np.sqrt(np.square(self.all_positions[cur_scene] - cur_position).sum(axis=1).min())
        
        return min_diff <= 2

In [11]:
def check_bump(last_observations, observations):
    return np.sqrt(np.square(last_observations['agent_position'] - observations['agent_position']).sum()) <= 0.5


In [12]:
saver = Saver()

for i in range(len(env.episodes)):
    observations = env.reset()
    
    if not(check_is_good(observations['rgb'], env)):
        continue
        
    last = ''
    is_bump = False
    
    for i in range(300):
        action = get_random_action(last, is_bump)
        last = action
        
        last_observations = observations
        observations = env.step(action)
        
        is_bump = check_bump(last_observations, observations)
        
        saver.save_image(observations, env)
        

GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
Terrible
NavigationEpisode(episode_id='10363', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[5.079437255859375, 0.09678955376148224, 1.7652792930603027], start_rotation=[0, 0.9992944925387229, 0, 0.03755685266068333], info={'geodesic_distance': 4.002524375915527, 'difficulty': 'easy'}, goals=[NavigationGoal(position=[8.691902160644531, 0.09678955376148224, 1.9023568630218506], radius=None)], start_room=None, shortest_paths=None)
GOOD!
GOOD!
GOOD!
GOOD!
Terrible
NavigationEpisode(episode_id='55042', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[5.738278388977051, 0.09678955376148224, 1.8620188236236572], start_rotation=[0, 0.8157571298766039, 0, 0.5783945928650144], info={'geodesic_distance': 10.994686126708984, 'difficulty': 'hard'}, goals=[NavigationGoal(position=[0.28938770294189453, 0.09678955376148224, 0.164236918091774], radius=None)], start_room=None, shortest_paths=None)
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!

Terrible
NavigationEpisode(episode_id='65897', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[5.095452308654785, 0.09678955376148224, 1.5962375402450562], start_rotation=[0, 0.9900050257212951, 0, 0.14103208516709137], info={'geodesic_distance': 6.063809394836426, 'difficulty': 'medium'}, goals=[NavigationGoal(position=[4.7055206298828125, 0.09678955376148224, -3.6890830993652344], radius=None)], start_room=None, shortest_paths=None)
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
Terrible
NavigationEpisode(episode_id='31254', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[6.157083511352539, 0.09678955376148224, 1.279064416885376], start_rotation=[0, 0.995245319608878, 0, -0.09739996815514007], info={'geodesic_distance': 7.823722839355469, 'difficulty': 'medium'}, goals=[NavigationGoal(position=[2.9760689735412598, 0.09678955376148224, -4.783332347869873], radius=None)], start_room=None, shortest_

GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
Terrible
NavigationEpisode(episode_id='44235', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[5.352810382843018, 0.09678955376148224, 0.9379595518112183], start_rotation=[0, 0.9588385948676937, 0, -0.28395166664794674], info={'geodesic_distance': 11.698190689086914, 'difficulty': 'hard'}, goals=[NavigationGoal(position=[-0.5678128004074097, 0.09678955376148224, -7.045673370361328], radius=None)], start_room=None, shortest_paths=None)
GOOD!
GOOD!
GOOD!
Terrible
NavigationEpisode(episode_id='7773', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[6.255983829498291, 0.09678955376148224, 1.1654869318008423], start_rotation=[0, 0.735791011990249, 0, 0.6772086729172666], info={'geodesic_distance': 8.867180824279785, 'difficulty': 'hard'}, goals=[NavigationGoal(position=[3.95609712600708, 0.09678955376148224, -5.675692558288574], radius=None)], start_room=None, sho

GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
Terrible
NavigationEpisode(episode_id='64710', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[5.736767292022705, 0.09678955376148224, 1.1612699031829834], start_rotation=[0, 0.8517911547846033, 0, 0.523881502470465], info={'geodesic_distance': 10.835487365722656, 'difficulty': 'hard'}, goals=[NavigationGoal(position=[0.2099347561597824, 0.09678955376148224, 0.6034982204437256], radius=None)], start_room=None, shortest_paths=None)
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
Terrible
NavigationEpisode(episode_id='24394', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[5.560756206512451, 0.09678955376148224, 1.8172287940979004], start_rotation=[0, 0.5369437833883297, 0, 0.8436180258153725], info={'geodesic_distance': 6.858005523681641, 'difficulty': 'medium'}, goals=[NavigationGoal(position=[2.571285009

GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
Terrible
NavigationEpisode(episode_id='41901', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[7.374203205108643, 0.09678955376148224, 1.4670780897140503], start_rotation=[0, 0.7573468466014751, 0, 0.6530128283141164], info={'geodesic_distance': 12.613302230834961, 'difficulty': 'hard'}, goals=[NavigationGoal(position=[-1.1861757040023804, 0.09678955376148224, 1.1992496252059937], radius=None)], start_room=None, shortest_paths=None)
GOOD!
Terrible
NavigationEpisode(episode_id='611', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[7.06553840637207, 0.09678955376148224, 0.1448061168193817], start_rotation=[0, 0.9999597929932628, 0, 0.008967295962053149], info={'geodesic_distance': 10.609314918518066, 'difficulty': 'hard'}, goals=[NavigationGoal(position=[-2.169853448867798, 0.09678955376148224, -2.545083999633789], radius=None)], start_room=None, shortest_path

GOOD!
GOOD!
GOOD!
Terrible
NavigationEpisode(episode_id='12043', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[6.824573516845703, 0.09678955376148224, 0.316313773393631], start_rotation=[0, 0.7912123415876942, 0, -0.6115415198654283], info={'geodesic_distance': 5.405932426452637, 'difficulty': 'easy'}, goals=[NavigationGoal(position=[2.563995838165283, 0.09678955376148224, -1.8099403381347656], radius=None)], start_room=None, shortest_paths=None)
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
Terrible
NavigationEpisode(episode_id='32926', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[5.726015567779541, 0.09678955376148224, -0.3370420038700104], start_rotation=[0, 0.8675238986999836, 0, -0.49739550177336816], info={'geodesic_distance': 11.282374382019043, 'difficulty': 'hard'}, goals=[NavigationGoal(position=[-2.2297065258026123, 0.09678955376148224, 0.9912753105163574], radius=None)], start_room=None, shortest_paths=None)
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
G

NavigationEpisode(episode_id='59594', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[4.365411281585693, 0.09678955376148224, -6.592906951904297], start_rotation=[0, 0.17085022116812423, 0, -0.9852970120358647], info={'geodesic_distance': 3.8358373641967773, 'difficulty': 'easy'}, goals=[NavigationGoal(position=[6.183778762817383, 0.09678955376148224, -5.916940212249756], radius=None)], start_room=None, shortest_paths=None)
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
Terrible
NavigationEpisode(episode_id='42673', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[8.355635643005371, 0.09678955376148224, 0.958378791809082], start_rotation=[0, 0.8371027797255733, 0, 0.5470456435944977], info={'geodesic_distance': 2.4898481369018555, 'difficulty': 'easy'}, goals=[NavigationGoal(position=[6.943402290344238, 0.09678955376148224, -0.8288301825523376], radius=None)], start_roo

GOOD!
GOOD!
GOOD!
Terrible
NavigationEpisode(episode_id='8753', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[6.978873252868652, 0.09678955376148224, -0.7813370227813721], start_rotation=[0, 0.8104845979381339, 0, -0.5857599478498521], info={'geodesic_distance': 8.155412673950195, 'difficulty': 'hard'}, goals=[NavigationGoal(position=[5.512768268585205, 0.09678955376148224, -4.941608905792236], radius=None)], start_room=None, shortest_paths=None)
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
GOOD!
Terrible
NavigationEpisode(episode_id='66900', scene_id='data/scene_datasets/gibson/Kerrtown.glb', start_position=[6.486849308013916, 0.09678955376148224, -0.4454273581504822], start_rotation=[0, 0.9973073478573588, 0, -0.07333521602696187], info={'geodesic_distance': 2.907597303390503, 'difficulty': 'easy'}, goals=[NavigationGoal(position=[4.257855415344238, 0.09678955376148224, -1.6170947551727295], radius=None)], start_room=None, shortest_paths=None)
GOO

KeyboardInterrupt: 